[View in Colaboratory](https://colab.research.google.com/github/youqad/Neurorobotics_Project/blob/master/Inferring_Space_from_Sensorimotor_Dependencies.ipynb)

# Inferring Space from Sensorimotor Dependencies

[![Documentation Status](https://readthedocs.org/projects/neurorobotics-project/badge/?version=latest)](https://neurorobotics-project.readthedocs.io/en/latest/?badge=latest)

# Simulations


In [1]:
# /!\ We're using Python 3

import plotly
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go

import matplotlib.pyplot as plt
from scipy.spatial import distance

from sklearn import preprocessing

from __future__ import division

%matplotlib inline

plotly.offline.init_notebook_mode(connected=True)
from IPython.core.display import display, HTML, Markdown
# The polling here is to ensure that plotly.js has already been loaded before
# setting display alignment in order to avoid a race condition.
display(HTML(
    '<script>'
        'var waitForPlotly = setInterval( function() {'
            'if( typeof(window.Plotly) !== "undefined" ){'
                'MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });'
                'MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);'
                'clearInterval(waitForPlotly);'
            '}}, 250 );'
    '</script>'
))

# Colorscales
def colorscale_list(cmap, number_colors, return_rgb_only=False):
    cm = plt.get_cmap(cmap)
    colors = [np.array(cm(i/number_colors)) for i in range(1, number_colors+1)]
    rgb_colors_plotly = []
    rgb_colors_only = []
    for i, c in enumerate(colors):
        col = 'rgb{}'.format(tuple(255*c[:-1]))
        rgb_colors_only.append(col)
        rgb_colors_plotly.append([i/number_colors, col])
        rgb_colors_plotly.append([(i+1)/number_colors, col])
    return rgb_colors_only if return_rgb_only else rgb_colors_plotly

from scipy.io import loadmat, whosmat
from numpy.random import randint

def formatted(f): 
    return format(f, '.2f').rstrip('0').rstrip('.')

## Organism 1

**Parameter**|**Value**
-|-
Dimension of motor commands|$40$
Dimension of environmental control vector|$40$
Dimension of proprioceptive inputs|$16 \quad (= 4×4)$
Dimension of exteroceptive inputs|$40 \quad (= 2 × 20)$
Number of eyes|$2$
Number of joints|$4$
Diaphragms|None
Number of lights|$3$
Light luminance|Fixed

In [21]:
!pip install --force-reinstall git+https://github.com/youqad/Neurorobotics_Project.git#egg=sensorimotor_dependencies

  Cloning https://github.com/youqad/Neurorobotics_Project.git to /tmp/pip-install-z8zwer1s/sensorimotor-dependencies
  Running setup.py bdist_wheel for sensorimotor-dependencies ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-rpxz67ih/wheels/55/44/c5/17ebe4a63673ffe2fd1d70e35a212ad7bf3302b4795b7348e5
Successfully built sensorimotor-dependencies
  Found existing installation: sensorimotor-dependencies 0.0.1
    Can't uninstall 'sensorimotor-dependencies'. No files were found to uninstall.


In [0]:
from sensorimotor_dependencies import organisms

organisms.Organism1(proprio=1, nb_joints=2, extero=1, nb_eyes=2).get_proprioception()

In [0]:
#@title Parameters

M_size = 40 #@param {type:"slider", min:10, max:100, step:5}
E_size = 40 #@param {type:"slider", min:10, max:100, step:5}

# Number of Joints / q
nb_joints = 4 #@param {type:"slider", min:3, max:5, step:1}

# Number of eyes / p
nb_eyes = 2 #@param {type:"slider", min:2, max:5, step:1}

# Number of lights / r
nb_lights = 3 #@param {type:"slider", min:2, max:5, step:1}

# Number of exteroceptive photosensors / p'
extero = 20 #@param {type:"slider", min:15, max:25, step:1}

# Number of proprioceptive sensors / q'
proprio = 4 #@param {type:"slider", min:3, max:5, step:1}


# Sensory inputs were generated from...
nb_generating_motor_commands = 50 #@param {type:"slider", min:10, max:200, step:10}
nb_generating_env_positions = 50 #@param {type:"slider", min:10, max:200, step:10}

# Neighborhood size of the linear approximation:
# Motor commands/Environmental positions drawn from normal distribution
# with mean zero and standard deviation... 
neighborhood_size = 1e-8
# (Coordinates differing from 0 by more than the std deviation are set equal to 0)

sigma = np.tanh

In [25]:
seed = 3
sim = []

for var in np.arange(.1, 1, .1):
  val, bod, env, env_bod = organisms.Organism1(seed=seed, retina_size=var).get_dimensions()
  print("var = {:.1f} / dim of compensated mov. = {} ({} {} {})".format(var, val, bod, env, env_bod))
  sim.append(val)

sim = np.array(sim)
best_sim = np.argmax(sim)

var = 0.1 / dim of compensated mov. = 0 (5 1 6)
var = 0.2 / dim of compensated mov. = 0 (5 1 6)
var = 0.3 / dim of compensated mov. = 0 (5 1 6)
var = 0.4 / dim of compensated mov. = 0 (5 1 6)
var = 0.5 / dim of compensated mov. = 0 (5 1 6)
var = 0.6 / dim of compensated mov. = 0 (5 1 6)
var = 0.7 / dim of compensated mov. = 0 (5 1 6)
var = 0.8 / dim of compensated mov. = 0 (5 1 6)
var = 0.9 / dim of compensated mov. = 0 (5 1 6)


In [0]:
# Other Dimension Reduction Methods
## LDA is not applicable for our data
## Found codes for classical MDS so far -- 
## to be cordinated with our data


#Autoencoder: https://github.com/asdspal/dimRed/blob/master/autoencoder.ipynb